# TensorFlow Softmax MNIST Regression
* Adapted from [TensorFlow-Examples](https://github.com/aymericdamien/TensorFlow-Examples/)

In [9]:
import os
import tensorflow as tf
from datetime import *

# clearing graph
tf.reset_default_graph()

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [10]:
# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 100
display_epoch = 1
unique = datetime.now().strftime('%m-%d_%H_%M')
logs_path = os.path.join('./logs/', unique)
export_path = os.path.join('./model/', unique)

# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

In [11]:
# Construct model and encapsulating all ops into scopes, making
# Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    #pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
    pred = tf.matmul(x, W) + b # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
    
    # Minimize error with *better* cross entropy
    # cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
    
    # Minimize error using squared error
    # cost = tf.nn.l2_loss(y - pred)
    
    # Minimize with computed square error
    cost = tf.reduce_mean(tf.pow(y - pred, 2))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [12]:
# Start Training
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_epoch == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))
    
    # saving model
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)
    builder.add_meta_graph_and_variables(sess, 
                                         [tf.saved_model.tag_constants.SERVING], 
                                         signature_def_map=None,
                                         assets_collection=None)
    builder.save()
    print("Model saved!")

Epoch: 0001 cost= 0.058085646
Epoch: 0002 cost= 0.046461421
Epoch: 0003 cost= 0.044299498
Epoch: 0004 cost= 0.043249138
Epoch: 0005 cost= 0.042596004
Epoch: 0006 cost= 0.042126678
Epoch: 0007 cost= 0.041768017
Epoch: 0008 cost= 0.041478614
Epoch: 0009 cost= 0.041240877
Epoch: 0010 cost= 0.041035947
Epoch: 0011 cost= 0.040862761
Epoch: 0012 cost= 0.040709284
Epoch: 0013 cost= 0.040575393
Epoch: 0014 cost= 0.040456640
Epoch: 0015 cost= 0.040347769
Epoch: 0016 cost= 0.040253084
Epoch: 0017 cost= 0.040163960
Epoch: 0018 cost= 0.040081480
Epoch: 0019 cost= 0.040010688
Epoch: 0020 cost= 0.039943084
Optimization Finished!
Accuracy: 0.8622
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'./model/01-03_17_05\\saved_model.pb'
Model saved!
